## Responsavel por tratar os dados extraidos da internet

In [29]:
dicionario_renomear_colunas = {
    "PM 2,5": "PM2.5",
    "Partículas Respiráveis (<2,5µm)(µg/m3)": "PM2.5",
    "Partículas Inaláveis (<10µm)(µg/m3)": "PM10",
    "Temperatura (ºC)": "Temperatura",
    "Tº": "Temperatura",
    "Radiação Solar (W/m2)": "Radiação Solar",
    "Monóxido de Carbono (ppm)": "Monóxido de Carbono",
    "CO": "Monóxido de Carbono",
    "Velocidade Escalar do Vento (m/s)": "Velocidade do Vento",
    "Direção Escalar do Vento (grau)": "Direção do Vento",
    "Umidade relativa (%)": "Umidade Relativa",
    "Precipitação Pluviométrica (mm)": "Precipitação",
    "Pressão Atmosférica (mmHg)": "Pressão Atmosférica",
    "Dióxido de enxofre (ppb)": "Dióxido de Enxofre",
    "Ozônio (ppb)": "Ozônio",
    "Monóxido de Nitrogênio (ppb)": "Monóxido de Nitrogênio",
    "Óxidos de Nitrogênio (ppb)": "Óxidos de Nitrogênio",
    "Dióxido de Nitrogênio (ppb)": "Dióxido de Nitrogênio",
    "HCT (ppm)": "HCT",
    "CH4 (ppm)": "Metano",
    "HCNM (ppm)": "HCNM",
    "Monóxido de Carbono Média Móvel (ppm)": "Monóxido de Carbono Média Móvel",
    "Data": "Data",
    "Hora": "Hora",
    "Média Horária": "Média Horária",
    "Seno da direção do vento": "Seno da Direção do Vento",
    "Cosseno da direção do vento": "Cosseno da Direção do Vento",
    "Seno ponderado": "Seno Ponderado",
    "Cosseno ponderado": "Cosseno Ponderado",
    "PM10": "PM10",
    "NO2": "Dióxido de Nitrogênio",
    "O3": "Ozônio",
    "SO2": "Dióxido de Enxofre",
    "DV": "Direção do Vento",
    "PP": "Pressão Atmosférica",
    "PA": "Precipitação",
    "VV": "Velocidade do Vento",
    "RS": "Radiação Solar",
    "UR": "Umidade Relativa"
}

In [30]:
import os
import pandas as pd
import numpy as np

pasta_principal = "../data"
pastas_estacoes = [f for f in os.listdir(pasta_principal) if os.path.isdir(os.path.join(pasta_principal, f))]


def obter_skiprows(ano, estacao):
    if ano == 2021 and estacao == "Cascata":
        return [0, 1, 3]
    elif ano >= 2015:
        return [0, 2]
    else:
        return None


for pasta_estacao in reversed(pastas_estacoes):
    path = os.path.join(pasta_principal, pasta_estacao)
    dados = [f for f in os.listdir(path) if f.endswith(".xlsx")]

    for arquivo in dados:
        ano = int(arquivo.split("_")[1].split(".")[0])
        caminho_arquivo = os.path.join(path, arquivo)
        skiprows = obter_skiprows(ano, pasta_estacao)
        df = pd.read_excel(caminho_arquivo, skiprows=skiprows)

        if ano < 2015:
            df['Data'] = pd.to_datetime(df['Data'], format='%d/%m/%Y', errors='coerce')
            df['Hora'] = pd.to_datetime(df['Hora'], format='%H:%M:%S', errors='coerce').dt.time

            # Combinar 'Data' e 'Hora' em 'Data e Hora'
            df['Data e Hora'] = pd.to_datetime(df['Data'].astype(str) + ' ' + df['Hora'].astype(str), errors='coerce')
            df = df.drop(columns=['Data', 'Hora'])
        else:
            df.rename(columns={df.columns[0]: 'Data e Hora'}, inplace=True)
            df['Data e Hora'] = pd.to_datetime(df['Data e Hora'], format='%d-%m-%Y %H:%M:%S', errors='coerce')

        df = df[df['Data e Hora'].notna()]
        df.set_index('Data e Hora', inplace=True)

        # Substituir valores inválidos
        pd.set_option('future.no_silent_downcasting', True)
        df.replace(['*', ' *', 'InVld', 'NoDat', 'Samp<', 'Alarm', 'Zero', 'map'], np.nan, inplace=True)
        # Renomear colunas
        colunas_validas = df.columns.intersection(dicionario_renomear_colunas.keys())
        df.rename(columns={col: dicionario_renomear_colunas[col] for col in colunas_validas}, inplace=True)

        # Verificar a presença da coluna 'PM2.5' antes de acessar
        if 'PM2.5' in df.columns:
            pm25_ausentes = df['PM2.5'].isna().sum()
            total_pm25 = len(df)
            percentual_ausentes = pm25_ausentes / total_pm25
            print(
                f"Estação: {pasta_estacao}, Ano: {ano}, Dados ausentes de PM2.5: {pm25_ausentes}/{total_pm25} ({percentual_ausentes:.2%})")
        else:
            print(f"Estação: {pasta_estacao}, Ano: {ano}, Coluna 'PM2.5' não encontrada.")

        # Salvar o arquivo tratado em uma pasta separada
        pasta_tratada = os.path.join("dados_tratados", "separado", pasta_estacao)
        os.makedirs(pasta_tratada, exist_ok=True)
        caminho_arquivo_tratado = os.path.join(pasta_tratada, f"{pasta_estacao}_{ano}_tratado.csv")
        df.to_csv(caminho_arquivo_tratado)

    # Combinar os dados tratados de todos os anos para cada estação
    pasta_tratada = os.path.join("dados_tratados", "separado", pasta_estacao)
    arquivos_tratados = [f for f in os.listdir(pasta_tratada) if f.endswith("_tratado.csv")]

    dfs_tratados = []
    for arquivo_tratado in arquivos_tratados:
        caminho_arquivo_tratado = os.path.join(pasta_tratada, arquivo_tratado)
        df_tratado = pd.read_csv(caminho_arquivo_tratado, index_col='Data e Hora', parse_dates=True)
        dfs_tratados.append(df_tratado)

    dados_tratados_combinados = pd.concat(dfs_tratados, ignore_index=False, axis=0)
    dados_tratados_combinados.reset_index(inplace=True)

    pasta_combinada = os.path.join("dados_tratados", "combinado", pasta_estacao)
    os.makedirs(pasta_combinada, exist_ok=True)
    dados_tratados_combinados.to_csv(os.path.join(pasta_combinada, f"{pasta_estacao}_tratado_combinado.csv"),
                                     index=False)


Estação: Piratininga, Ano: 2009, Dados ausentes de PM2.5: 8760/8760 (100.00%)
Estação: Piratininga, Ano: 2010, Dados ausentes de PM2.5: 8760/8760 (100.00%)
Estação: Piratininga, Ano: 2011, Dados ausentes de PM2.5: 8760/8760 (100.00%)
Estação: Piratininga, Ano: 2012, Dados ausentes de PM2.5: 8784/8784 (100.00%)
Estação: Piratininga, Ano: 2013, Dados ausentes de PM2.5: 8760/8760 (100.00%)
Estação: Piratininga, Ano: 2014, Dados ausentes de PM2.5: 8760/8760 (100.00%)
Estação: Piratininga, Ano: 2015, Dados ausentes de PM2.5: 7635/8760 (87.16%)
Estação: Piratininga, Ano: 2016, Dados ausentes de PM2.5: 1248/8784 (14.21%)
Estação: Piratininga, Ano: 2017, Dados ausentes de PM2.5: 1313/8760 (14.99%)
Estação: Piratininga, Ano: 2018, Dados ausentes de PM2.5: 1299/8760 (14.83%)
Estação: Piratininga, Ano: 2019, Dados ausentes de PM2.5: 1316/8760 (15.02%)
Estação: Piratininga, Ano: 2020, Dados ausentes de PM2.5: 989/8784 (11.26%)
Estação: Piratininga, Ano: 2021, Dados ausentes de PM2.5: 564/8760 (6.4